In [1]:
import json
import pickle

import numpy as np
import pandas as pd
import torch
from scipy import stats

with open("predictions/y_pred_reg_2000.pkl", "rb") as f:
    reg_preds = pickle.load(f)
with open("predictions/class_predictions.pkl", "rb") as f:
    class_preds = pickle.load(f)

In [16]:
reg_preds[0]

[tensor([[9.2709]], dtype=torch.float64),
 tensor([[8.9380]], dtype=torch.float64),
 tensor([[8.5971]], dtype=torch.float64),
 tensor([[9.2003]], dtype=torch.float64),
 tensor([[8.2386]], dtype=torch.float64),
 tensor([[8.0336]], dtype=torch.float64),
 tensor([[8.9644]], dtype=torch.float64),
 tensor([[8.7292]], dtype=torch.float64),
 tensor([[8.5579]], dtype=torch.float64),
 tensor([[2.8362]], dtype=torch.float64)]

In [3]:
# Concatenate the tensors along the appropriate dimension
flattened_tensor_list = [item for sublist in reg_preds for item in sublist]

In [11]:
preds = []
for j in range(len(flattened_tensor_list)):
    if isinstance(flattened_tensor_list[j], bool):
        preds.append(flattened_tensor_list[j])
        continue
    preds.append(flattened_tensor_list[j].numpy()[0][0])

In [27]:
y_preds_10 = np.array(preds).reshape(-1, 10)

# Calculate the Z-scores
z_scores = stats.zscore(
    y_preds_10, axis=1
)  # Calculate Z-scores for the entire array

# Create a boolean mask where elements with |Z-score| > 3 are True
outliers_mask = np.abs(z_scores) > 2.5

# Convert outliers to False
y_preds_10[outliers_mask] = False

print(y_preds_10)

[[ 9.27094227  8.93795492  8.59709037 ...  8.72924028  8.5579202
   2.83623753]
 [ 1.82612505  1.62816567  1.94056469 ...  2.50960277  1.6865316
   1.90979697]
 [-3.33576015 -5.03238398 -4.62953495 ... -3.27101134 -0.11896646
  -6.32220861]
 ...
 [-3.40987781 -8.0824008  -8.22443831 ... -9.22857166 -5.2933497
  -5.26144869]
 [16.52339519 16.46747191 16.45167346 ... 16.30263355 16.50970616
  16.0322107 ]
 [12.60661496 12.43208853 13.08309928 ... 12.39090511 12.53098172
  12.74558435]]
[[ 9.27094227  8.93795492  8.59709037 ...  8.72924028  8.5579202
   0.        ]
 [ 1.82612505  1.62816567  1.94056469 ...  2.50960277  1.6865316
   1.90979697]
 [-3.33576015 -5.03238398 -4.62953495 ... -3.27101134  0.
  -6.32220861]
 ...
 [-3.40987781 -8.0824008  -8.22443831 ... -9.22857166 -5.2933497
  -5.26144869]
 [16.52339519 16.46747191 16.45167346 ... 16.30263355 16.50970616
   0.        ]
 [12.60661496 12.43208853 13.08309928 ... 12.39090511 12.53098172
  12.74558435]]


In [33]:
y_preds_10[y_preds_10 == 0] = None

# Calculate the mean of each row, ignoring None values (use np.nanmean)
row_means = np.nanmean(y_preds_10.astype(float), axis=1)
row_stds = np.nanstd(y_preds_10.astype(float), axis=1)
class_preds = class_preds[:2000]

C:\Users\je009447\AppData\Local\Temp\ipykernel_19332\2308121848.py:4: RuntimeWarning: Mean of empty slice
  row_means = np.nanmean(y_preds_10.astype(float), axis=1)
c:\Users\je009447\AppData\Local\miniconda3\envs\django_outlierdetection\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [52]:
output_data = {}
for i in range(len(row_means)):

    output_data[i] = {
            #"class": np.int64(class_preds[i]),
            "class_conf": 1,
            "pdf_type": "norm",
            "pdf_args": {"loc": row_means[i], "scale": row_stds[i]},
    }

In [53]:
output_data[0]["class_conf"].dtype

AttributeError: 'int' object has no attribute 'dtype'

In [54]:
json_file_path = "submission.json"

# Save dictionary to JSON file
with open(json_file_path, "w") as json_file:
    json.dump(output_data, json_file, indent=4)